In [2]:
! nvidia-smi

Mon Aug  9 16:35:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
! pip install stable_baselines3
! pip install espnet
! pip install espnet_model_zoo

     |████████████████████████████████| 172 kB 8.2 MB/s 
     |████████████████████████████████| 787 kB 9.5 MB/s 
     |████████████████████████████████| 124 kB 66.3 MB/s 
     |████████████████████████████████| 636 kB 58.2 MB/s 
     |████████████████████████████████| 1.5 MB 52.7 MB/s 
     |████████████████████████████████| 212 kB 78.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174 kB 62.3 MB/s 
     |████████████████████████████████| 86 kB 7.5 MB/s 
     |████████████████████████████████| 1.0 MB 61.4 MB/s 
     |████████████████████████████████| 69 kB 10.4 MB/s 
     |████████████████████████████████| 13.1 MB 132 kB/s 
     |████████████████████████████████| 1.8 MB 62.2 MB/s 
     |████████████████████████████████| 1.3 MB 51.3 MB/s 
     |████████████████████████████████| 241 kB 68.3 MB/s 
     |████████████████████████████████| 3.1 MB 47.0 MB/s 
     |███

In [4]:
import numpy as np
import random
import time

# libraries for RL
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3 import PPO, DQN, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

# libraries for ASR
import soundfile
from espnet2.bin.asr_inference import Speech2Text
from espnet_model_zoo.downloader import ModelDownloader
import string

from stable_baselines3.common.env_checker import check_env

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [5]:
FOODS = ('NO', 'APPLE', 'ORANGE')
# 'no' is a special food that contains no nuturition
# each food contains different amount of nutrition (nutrition A and B)
NUTRITIONA = (0, 2, 1)
NUTRITIONB = (0, 1, 2)

# definition of food. It has name, image, and two types of nuturition A and B
class Food():
    def __init__(self, name, image, nutritionA, nutritionB):
        self.name = name
        self.image = image
        self.nutritionA = nutritionA
        self.nutritionB = nutritionB

In [6]:
# Prepare ASR

# download and build models
# tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave'
# tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave'
# tag = 'Shinji Watanabe/gigaspeech_asr_train_asr_raw_en_bpe5000_valid.acc.ave'
# tag = 'Shinji Watanabe/gigaspeech_asr_train_asr_raw_en_bpe5000_valid.acc.ave'
tag = 'kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave'
# tag = 'kamo-naoyuki/librispeech_asr_train_asr_conformer5_raw_bpe5000_frontend_confn_fft512_frontend_confhop_length256_scheduler_confwarmup_steps25000_batch_bins140000000_optim_conflr0.0015_initnone_sp_valid.acc.ave'
d = ModelDownloader()
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

https://zenodo.org/record/4604066/files/asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave.zip?download=1: 100%|██████████| 650M/650M [00:50<00:00, 13.5MB/s]


In [8]:
# Dialogue partner for the language learning agent, which is the outside environment
class DialogWorld:
    MAX_STEP = 10

    def __init__(self):
        self.food_storage = list() # list of food that the environment has
        for f in FOODS:
            if f == 'no': continue
            for k in range(10):
                # we are using a scalar value as an 1x1 picel image.
                # The value of the pixcel is equal to the food ID added with small noise
                image = FOODS.index(f) + random.random()/5 # 1x1 image (dummy sample)
                nutA = NUTRITIONA[FOODS.index(f)]
                nutB = NUTRITIONB[FOODS.index(f)]
                self.food_storage.append(Food(f, image, nutA, nutB))
        self.reset()
        
    def reset(self):
        self.num = self.MAX_STEP
        self.leftfood = random.choice(self.food_storage)
        self.rightfood = random.choice(self.food_storage)
        self.done = False

    def step(self, action):
        nbests = speech2text(action)
        text, *_ = nbests[0]
        print(text)
        if text in FOODS:
            dlg_success = True
            foodID = FOODS.index(text)
        else:
            dlg_success = False
            foodID = -1
        self.num -= 1
        if self.num == 0:
            self.done = True
        return (dlg_success, foodID), self.done
            
    def observe(self):
        return np.array([self.num, self.leftfood.image, self.rightfood.image])


In [9]:
# Environment for RL.
# Internal state of spolacq agent is inside in this environment
class SpoLacq(gym.Env):
    # Agent has nutrition tanks A and B as its internal state
    MAX_NUTRITION = 50 # maximum nutrition tank capacity
    Full_nutritionA = 15 # ideal nutrition level of nutrition A
    Full_nutritionB = 10 # ideal nutrition level of nutrition B

    def __init__(self):
        super().__init__()
        self.dlgworld = DialogWorld()
        self.action_space = gym.spaces.Discrete(len(FOODS))
        # observation space : (nutA, nutB, remain_step, image1, image2)
        self.observation_space = gym.spaces.Box(low=0, high=max(self.MAX_NUTRITION, self.dlgworld.MAX_STEP, len(FOODS)), shape=(5,))
        # Read sound files for sound dictionary
        soundfiles = ['samples/wordwav/no-1.wav', 'samples/wordwav/apple-1.wav', 'samples/wordwav/orange-1.wav']
        self.sounddic = list() # convert categorical ID to wave utterance
        for f in soundfiles:
            utterance, rate = soundfile.read(f)
            assert rate == 16000, "mismatch in sampling rate"
            self.sounddic.append(utterance)
        self.reset()
        
    def reset(self):
        # Initialize the dialogue world
        self.dlgworld.reset()
        # Initialize the internal state of spolacq agent
        self.nutritionA = random.random()*5
        self.nutritionB = random.random()*5
        return self.observe()

    def step(self, action):
        old_state = self.observe()
        utterance = self.sounddic[action]
        feedback, dlg_done = self.dlgworld.step(utterance)
        self.update_internal_state(feedback)
        new_state = self.observe()
        reward = self.reward(old_state, new_state, feedback)
        return new_state, reward, dlg_done, {}
        
    def observe(self):
        insideobs = np.array([self.nutritionA, self.nutritionB])
        outsideobs = self.dlgworld.observe()
        observation = np.concatenate([insideobs, outsideobs])
        return observation
    
    def update_internal_state(self, feedback):
        if not feedback[0]: # failed dialogue
            pass
        else:
            self.nutritionA = NUTRITIONA[feedback[1]]
            self.nutritionB = NUTRITIONB[feedback[1]]
        if self.MAX_NUTRITION < self.nutritionA:
            self.nutritionA = self.MAX_NUTRITION
        if self.MAX_NUTRITION < self.nutritionB:
            self.nutritionB = self.MAX_NUTRITION
            
    def reward(self, old_state, new_state, feedback):
        if not feedback[0]: # failed dialogue
            return -10
        old_nutA = old_state[0]
        old_nutB = old_state[1]
        new_nutA = new_state[0]
        new_nutB = new_state[1]
        dlg_remaining_turn = new_state[2]
        old_dissatisfaction = abs(self.Full_nutritionA - old_nutA) + abs(self.Full_nutritionB - old_nutB)
        new_dissatisfaction = abs(self.Full_nutritionA - new_nutA) + abs(self.Full_nutritionB - new_nutB)
        diff_satisfaction = old_dissatisfaction - new_dissatisfaction
        if 0 < dlg_remaining_turn:
            return diff_satisfaction / 10
        else:
            return new_dissatisfaction
        
    def render(self, mode='console', close=False):
        state = self.observe()
        print(state)
        
    def close(self):
        pass

    def seed(self, seed=None):
        random.seed(seed)

In [10]:
def make_env(rank, seed=0):
    def _init():
        # env = gym.make("CartPole-v1")
        env = SpoLacq()
        env.seed(seed + rank)
        return env
    set_random_seed(seed)
    return _init



In [11]:
if __name__ == '__main__':

    # RL environment creation
    # env = Monitor(SpoLacq())
    # env = DummyVecEnv([lambda: env])

    # env = SubprocVecEnv([make_env(i) for i in range(2)], start_method='forkserver')
    env = DummyVecEnv([make_env(i) for i in range(2)])

    # RL learning model creation
    model = PPO('MlpPolicy', env, verbose=1)


    # RL model training
    model.learn(total_timesteps=100000)

    # Test the learnt agent
    state = env.reset()
    total_reward = 0
    while True:
        # render the state
        # env.render()

        # Agent gets an environment state and returns a decided action
        action, _ = model.predict(state)
        type(action)

        # Environment gets an action from the agent, proceeds the time step,
        # and returns the new state and reward etc.
        state, reward, done, info = env.step(action)
        total_reward += reward
        # print(action)
        print(FOODS[action[0]], reward)

        print('')
        if done.any():
            print('total_reward:', total_reward)
            break

RuntimeError: ignored